# O Problema: PNAD-COVID 19

Você foi contratado por um grande hospital para entender o comportamento da população na época da pandemia da COVID-19 e para definir quais indicadores seriam importantes para o planejamento.

O estudo do PNAD COVID-19 foi identificado como uma ótima base para extrair estas respostas, porém não será necessário utilizar todas as informações coletadas pelo IBGE. Basicamente, a análise deve abranger:

- Características clínicas dos sintomas;

- Características da população;

- Características econômicas da sociedade.

Além disso, alguns requisitos para a análise são:

- Utilizar no máximo 20 questionamentos realizados na pesquisa;

- Utilizar 3 meses de dados para construção da solução.

# Leitura e processamentos dos dados

Como fonte das informações, será utilizada a base de microdados do PNAD COVID-19 pois disponibiliza tabelas em formato .csv já parcialmente pré-processadas para análise de dados. A única desvantagem é a necessidade de "traduzir" os códigos das colunas para nomes que sejam compreensíveis, além de que os microdados são separados por mês de pesquisa, portanto será necessário concatenar os .csvs de três meses diferentes.

Para esta análise, as tabelas de microdados dos últimos três meses de pesquisa foram selecionadas, corresponde aos meses de setembro, outubro e novembro de 2020.

Fonte dos dados: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados

In [2]:
import pandas as pd
import os

# Leitura dos arquivos brutos

In [3]:
data_folder = './data/'

pnad_files_path = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.startswith('PNAD')]
pnad_files_path

['./data/PNAD_COVID_092020.csv',
 './data/PNAD_COVID_102020.csv',
 './data/PNAD_COVID_112020.csv']

In [4]:
pnad = pd.read_csv(pnad_files_path[0])

In [6]:
print(f'Colunas: {len(pnad.columns)}')
print(f'Linhas: {len(pnad)}')
display(pnad.head())

Colunas: 145
Linhas: 387298


,Ano,UF,CAPITAL,RM_RIDE,V1008,V1012,V1013,V1016,Estrato,UPA,...,F001,F0021,F0022,F002A1,F002A2,F002A3,F002A4,F002A5,F0061,F006
0,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
1,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
2,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
3,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
4,2020,11,11.0,NaN,2,1,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0


Tem-se então um total de 145 colunas para cada linha do DataFrame. Como especificado no problema, deve-se reduzir este número para algo em torno de 20 colunas.

Para selecionar as colunas mais valiosas, o dicionário dos dados deve ser consultado.

Dicionário dos dados: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao

# Seleção de colunas

**Dicionário de dados das colunas mantidas no dataframe**

- Ano: ano de referência

- UF: Unidade da Federação

- V1013: Mês da pesquisa

- V1022: Situação do domicílio

- V1023: Tipo de área

- A002: idade do morador

- A003: sexo

- A004: Cor ou raça

- A005: escolaridade